In [2]:
import pandas as pd
import re 

'''def clean_company_name(name: str) -> str:
    """Normalize stock names to plain company names."""
    name = re.sub(r'\bCommon Stock\b', '', name, flags=re.IGNORECASE)
    name = re.sub(r'\bOrdinary Shares?\b', '', name, flags=re.IGNORECASE)
    name = re.sub(r'\bDepositary Shares?.*', '', name, flags=re.IGNORECASE)
    name = re.sub(r'\bClass [A-Z]\b', '', name, flags=re.IGNORECASE)
    name = re.sub(r'\bLtd\b', 'Limited', name, flags=re.IGNORECASE)
    name = re.sub(r'\s+', ' ', name).strip()
    return name'''

# Load both ticker lists
#nyse_df = pd.read_csv("nyse_screener.csv")
#nasdaq_df = pd.read_csv("nasdaq_screener.csv")
Top_100 = pd.read_csv("Top_100.csv")

# Combine into a single DataFrame
#all_tickers = pd.concat([nyse_df, nasdaq_df], ignore_index=True)

# Normalize column names just in case
#all_tickers.columns = [col.strip().lower() for col in all_tickers.columns]
Top_100.columns = [col.strip().lower() for col in Top_100.columns]

# Clean names
#all_tickers['clean_name'] = all_tickers['name'].apply(clean_company_name)

# Create dictionaries for fast lookups
ticker_to_name = dict(zip(Top_100['symbol'].str.upper(), Top_100['name']))
valid_tickers = set(ticker_to_name.keys())
company_names = [name.lower() for name in ticker_to_name.values()]
name_to_ticker = {name.lower(): symbol for symbol, name in ticker_to_name.items()}

In [3]:
#imports
import json
import pandas as pd
import spacy
import re
from tqdm import tqdm
import spacy
import pandas as pd
import spacy
import warnings
from rapidfuzz import process, fuzz

#model
nlp = spacy.load('en_core_web_sm')

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

#to be ran per comment
def extract_ner_entities(model, text, similarity_threshold=90):
    
    BLACKLIST = {'ev', 'covid', 'etf', 'nyse', 'sec', 'spac', 'fda', 'treasury', 'covid-19', 'rrsp', 'tfsa','fed', 'Reuters aggregated](https://www.streetinsider.com/Reuters'.lower()}
    doc = model(text)
    detected_companies = []

    #Detect companies via spaCy NER
    for ent in doc.ents:
        if ent.label_ == "ORG" and ent.text.lower() not in BLACKLIST:
            org_name = ent.text.strip()
            # Fuzzy match against official company names from csv file
            match, score, _ = process.extractOne(org_name.lower(), company_names, scorer=fuzz.token_sort_ratio)
            if score >= similarity_threshold:
                matched_ticker = name_to_ticker[match]
                canonical_name = ticker_to_name[matched_ticker]
                detected_companies.append(canonical_name)
            else:
                #keeps companies not in csv file maybe delete later
                detected_companies.append(org_name)

    # --- Match stock tickers in text ---
    for token in doc:
        token_text = token.text.strip()

        # Handle tickers with $ prefix, e.g. $AAPL
        if token_text.startswith("$"):
            token_text = token_text[1:]

        # Check if it’s a valid ticker symbol
        if token_text in valid_tickers:
            company_name = ticker_to_name.get(token_text)
            detected_companies.append(company_name)


    return list(set(detected_companies))


def get_dict_top_companies(dataset, column_name, top_companies=10):
    company_counter = dict()
    for companies in dataset[column_name]:
        for company in companies:
            if company in company_counter:  
                company_counter[company] += 1
            else:
                company_counter[company] = 1
    sorted_dict = dict(sorted(company_counter.items(), key=lambda x: x[1], reverse=True))
    top = dict()
    for company, count in list(sorted_dict.items())[:top_companies]:
        top[company] = count

    return top

#Use this on Emre's code. 
'''pre_processed_sub["Companies"] = pre_processed_sub["selftext"].apply(lambda x: extract_ner_entities(nlp, x, similarity_threshold=60))
print(get_dict_top_companies(pre_processed_sub, 'Companies'))'''

'pre_processed_sub["Companies"] = pre_processed_sub["selftext"].apply(lambda x: extract_ner_entities(nlp, x, similarity_threshold=60))\nprint(get_dict_top_companies(pre_processed_sub, \'Companies\'))'